In [1]:
from python_app.utils.gpu_info import fetch_gpu_parameters, verify_gpu_allocation
import numba as nb
import numpy as np
from numba import cuda

# Power measurements
- `Period`: Number of points (each one 2.5ns) in a single pulse sequence
- `N`: Number periods to look at
- `NP = Period * N`: Total number of points in the plot
- `Repetition (R)`: How many times `NP` is repeated

`TOTAL_NUMBER_OF_POINTS = R*NP`

In [7]:
derived_parameters["STORAGE_PRECISION"]

4

In [10]:
type(np.int32)

type

In [8]:
raw_paramters = {
    "RECORD_LENGTH": 65536, # Read this parameter from SP-devices
    "DEVICE_PRECISION": 14, # 14 bit precision from -ve to +ve # constant
    "PROCESSING_ARRAY_TYPE": np.int32,
    "INPUT_ARRAY_TYPE": np.int16,
    "OUTPUT_ARRAY_TYPE": np.float32,
    "N": 3,
    "PERIOD_IN_NS": 200
}

derived_parameters = {}
derived_parameters["MAX_DEVICE_CODE"] = 2**(raw_paramters["DEVICE_PRECISION"] - 1)

derived_parameters["STORAGE_PRECISION"] = np.dtype(raw_paramters['PROCESSING_ARRAY_TYPE']).itemsize
derived_parameters["MAX_STORE_CODE"] = 2**(derived_parameters["STORAGE_PRECISION"] * 8 - 1)

derived_parameters["P"] = int(raw_paramters["PERIOD_IN_NS"] // 2.5 + 1)
derived_parameters["NP"] = raw_paramters["N"] * derived_parameters["P"]
derived_parameters["R"] = raw_paramters["RECORD_LENGTH"] // derived_parameters["NP"]
derived_parameters["TOTAL_POINTS"] = derived_parameters["R"] * derived_parameters["NP"]

total_paramters = {
    **raw_paramters, **derived_parameters
}

max_summation = total_paramters["MAX_DEVICE_CODE"] * total_paramters["R"]

if max_summation > total_paramters["MAX_STORE_CODE"]:
    raise RuntimeError(
        f"Max summation over repetitions (={max_summation} bytes) cannot be stored using (STORAGE_PRECISION={total_paramters['STORAGE_PRECISION']})"
    )

In [31]:
total_paramters

{'RECORD_LENGTH': 65536,
 'DEVICE_PRECISION': 14,
 'STORAGE_TYPE': int32,
 'N': 3,
 'PERIOD_IN_NS': 200,
 'MAX_DEVICE_CODE': 8192,
 'STORAGE_PRECISION': 32,
 'MAX_STORE_CODE': 2147483648,
 'P': 81,
 'NP': 243,
 'R': 269,
 'TOTAL_POINTS': 65367}

In [ ]:
BLOCKS_PER_GRID = (NUMBER_OF_FIELD_POINTS_PER_RUN, NUMBER_OF_FIELD_POINTS_PER_RUN)

potential_evaluator = PotentialEvaluator(NUMBER_OF_PHI_POINTS, potential_function_cuda)
THREADS_PER_BLOCK_potential_evaluation = allocate_max_threads(8)

In [3]:
from utils.info import gpu_check

gpu_check()

{'max_shared_memory_per_block': 49152,
 'max_threads_per_block': 1024,
 'max_block_dim_x': 1024,
 'max_block_dim_y': 1024,
 'max_block_dim_z': 64,
 'max_grid_dim_x': 2147483647,
 'max_grid_dim_y': 65535,
 'max_grid_dim_z': 65535}

# Legacy

In [1]:
from utils.array_stacker import ArrayStacker
from kernels.potential_minimum_searcher import PotentialMinimumSearcher
from kernels.potential_evaluator import PotentialEvaluator
from functions.potential import potential_function_cuda
from common import plotter
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from numba.cuda.cudadrv.devicearray import DeviceNDArray
from numba import cuda
import numpy as np
import math
import itertools
from collections import defaultdict
from typing import Tuple, List, Dict
from utils.info import allocate_max_threads, verify_blocks_per_grid


pi = math.pi
sin = np.sin
cos = np.cos

plt.style.use('my_official')